In [ ]:
# PubChem_crawlingg code
import pandas as pd
import requests  # 导入requests包
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import urllib.parse
import json
from lxml import etree
import openpyxl

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.append(
    ['pubchem_id', 'Name', 'MoleF', 'synon', 'MoleW', 'Sequence', 'SVG', 'IUPACC', 'TowD', 'ThreeD', 'IUPACN', 'Inchi',
     'Inchi_key', 'CanSmi', 'IsoSmi', 'TPSA', 'Comple', 'XLogP3', 'HAC', 'HBDC', 'HBAC', 'RBC', 'ExactM', 'FormC',
     'PhyDes', 'MeltP', 'Solu', 'LogP', 'Drug_in', 'Pharm', 'Distrub', 'Toxi', 'Metab', 'BioHalf', 'PDID', 'Infor',
     'GrugB'])
sheet2 = wb.create_sheet('title')
sheet2.title = 'new sheet'
sheet1.sheet_properties.tabColor = '000000'
element = []
disable_warnings(InsecureRequestWarning)

def Pubmed_content(TOCHeading3):
    PD_url = TOCHeading3[k].get('URL')
    PD_res = requests.get(url=PD_url, headers=headers, verify=False)
    PD_res.encoding = PD_res.apparent_encoding
    tree = etree.HTML(PD_res.text)
    meta_list = tree.xpath('/html/head/meta')
    for meta in meta_list:
        pm_name = meta.xpath('./@name')
        if pm_name == ['log_op']:
            content = meta.xpath('./@content')
            if content == ['link']:
                PDID = tree.xpath('/html/head/meta[29]/@content')[0]
            elif content == ['retrieve']:
                PDID = tree.xpath('//*[@id="full-view-identifiers"]/li[1]/span/strong/text()')[0]
    return pm_name, PDID

if __name__ == '__main__':
    headers = {
        'User-Agent': "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/103.0.0.0 Mobile Safari/537.36 "
    }

    url_main = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/'
    # keyword = pd.read_excel('fileName.xlsx', dtype=object).loc[:, 'Column_Name']

    keyword = ['101063132', '10102206', '101290199', '101607771', '101241242', '101652762']  # Test data
    for i in range(len(keyword)):
        url = url_main + str(keyword[i]) + '/JSON/'
        print('https://pubchem.ncbi.nlm.nih.gov/compound/' + str(keyword[i]))
        PhyDes, MeltP, Solu, LogP, Drug_in, Pharm, Distrub = "", "", "", "", "", "", ""
        MoleF, Toxi, Metab, BioHalf, PDID, Infor, DrugB = "", "", "", "", "", "", ""
        synon, MoleW, Sequence, SVG, IUPACC, TowD, ThreeD = "", "", "", "", "", "", ""
        IUPACN, Inchi, Inchi_key, CanSmi, IsoSmi, TPSA = "", "", "", "", "", ""
        Comple, XLogP3, HAC, HBDC, HBAC, RBC, ExactM, FormC = "", "", "", "", "", "", "", ""
        response = requests.get(url=url, headers=headers, verify=False)
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        sect = data.get('Record').get('Section')
        Name = data.get('Record').get('RecordTitle')
        refer = data.get('Record').get('Reference')
        for m in range(len(refer)):
            SourceName = data.get('Record').get('Reference')[m].get('SourceName')
            if SourceName == 'DrugBank':
                DrugB = data.get('Record').get('Reference')[m].get('SourceID')
        for  j in range(len(sect)):
            TOCHeading = data.get('Record').get('Section')[j].get('TOCHeading')
            TOCHeading1 = sect[j].get('Section')
            if TOCHeading == 'Chemical and Physical Properties':
                TOCHeading1 = data.get('Record').get('Section')[j].get('Section')
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == 'Experimental Properties':
                        TOCHeading3 = TOCHeading1[j].get('Section')[k].get('Section')
                        for index in range(len(TOCHeading3)):
                            TOCHeading4 = TOCHeading3[index].get('TOCHeading')
                            value = TOCHeading3[index].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                            if TOCHeading4 == 'Physical Description':
                                PhyDes = value
                            elif TOCHeading4 == 'Melting Point':
                                MeltP = "have"
                            elif TOCHeading4 == 'Solubility':
                                Solu = value
                            elif TOCHeading4 == 'LogP':
                                LogP = value
                    if TOCHeading2 == 'Computed Properties':
                        TOCHeading3 = data.get('Record').get('Section')[j].get('Section')[k].get('Section')
                        for index in range(len(TOCHeading3)):
                            TOCHeading4 = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[
                                index].get(
                                'TOCHeading')
                            if TOCHeading4 == 'Molecular Weight':
                                MoleW = \
                                    data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                        'Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                            elif TOCHeading4 == 'XLogP3':
                                XLogP3 = \
                                    data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                        'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Hydrogen Bond Donor Count':
                                HBDC = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                    'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Hydrogen Bond Acceptor Count':
                                HBAC = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                    'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Rotatable Bond Count':
                                RBC = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                    'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Exact Mass':
                                ExactM = \
                                    data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                        'Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                            elif TOCHeading4 == 'Topological Polar Surface Area':
                                TPSA = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                    'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Heavy Atom Count':
                                HAC = data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                    'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Formal Charge':
                                FormC = \
                                    data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                        'Information')[0].get('Value').get('Number')[0]
                            elif TOCHeading4 == 'Complexity':
                                Comple = \
                                    data.get('Record').get('Section')[j].get('Section')[k].get('Section')[index].get(
                                        'Information')[0].get('Value').get('Number')[0]
            if TOCHeading == 'Biologic Description':
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == 'Biologic Depiction':
                        SVG = TOCHeading1[k].get('Information')[0].get('Value').get('ExternalDataURL')[0]
                        if SVG != "":
                            SVG = keyword[i] + '.svg'
                    if TOCHeading2 == 'Biologic Line Notation':
                        TOCHeading3 = TOCHeading1[k].get('Information')
                        for index in range(len(TOCHeading3)):
                            TOCHeading4 = TOCHeading3[index].get('Name')
                            if TOCHeading4 == 'IUPAC Condensed':
                                IUPACC = TOCHeading3[index].get('Value').get('StringWithMarkup')[0].get('String')
                            elif TOCHeading4 == 'Sequence':
                                Sequence = TOCHeading3[index].get('Value').get('StringWithMarkup')[0].get('String')
            if TOCHeading == 'Drug and Medication Information':
                TOCHeading2 = sect[j].get('Section')[0].get('TOCHeading')
                if TOCHeading2 == 'Drug Indication"':
                    Drug_in = sect[j].get('Section')[0].get('Information')
            if TOCHeading == 'Related Records':
                TOCHeading2 = sect[j].get('Section')[0].get('TOCHeading')
                if TOCHeading2 == "Entrez Crosslinks":
                    TOCHeading3 = sect[j].get('Section')[0].get('Information')
                    for k in range(len(TOCHeading3)):
                        TOCHeading4 = TOCHeading3[k].get('Name')
                        if TOCHeading4 == 'PubMed Count':
                            pm_name, PDID = Pubmed_content(TOCHeading3)
            if TOCHeading == 'Pharmacology and Biochemistry':
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == 'Pharmacology':
                        Pharm = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                    elif TOCHeading2 == 'Absorption':
                        Distrub = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                    elif TOCHeading2 == 'Metabolism/Metabolites':
                        Metab = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                    elif TOCHeading2 == 'Biological Half-Life':
                        BioHalf = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
            if TOCHeading == 'Toxicity':
                TOCHeading1 = sect[j].get('Section')
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == "Toxicological Information":
                        Toxi = TOCHeading1[k].get('Section')[0].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
            if TOCHeading == 'Names and Identifiers':
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == 'Record Description':
                        Infor = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                    elif TOCHeading2 == 'Molecular Formula':
                        MoleF = TOCHeading1[k].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String')
                    elif TOCHeading2 == 'Synonyms':
                        syn_list = TOCHeading1[k].get('Section')
                        for index in range(len(syn_list)):
                            syn_TOCHeading2 = TOCHeading1[k].get('Section')[index].get('TOCHeading')
                            if syn_TOCHeading2 == "Depositor-Supplied Synonyms":
                                synon_map = TOCHeading1[k].get('Section')[index].get('Information')[0].get('Value').get('StringWithMarkup')
                                synon = ""
                                for map in range(len(synon_map)):
                                    synon += TOCHeading1[k].get('Section')[index].get('Information')[0].get('Value').get('StringWithMarkup')[map].get('String') + "##"
                    elif TOCHeading2 == 'Computed Descriptors':
                        TOCHeading3 = TOCHeading1[k].get('Section')
                        for m in range(len(TOCHeading3)):
                            TOCHeading4 = TOCHeading3[m].get(
                                'TOCHeading')
                            if TOCHeading4 == "IUPAC Name":
                                IUPACN = str(TOCHeading3[m].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String'))
                            elif TOCHeading4 == "InChI":
                                Inchi = str(TOCHeading3[m].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String'))
                            elif TOCHeading4 == "InChIKey":
                                Inchi_key = str(TOCHeading3[m].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String'))
                            elif TOCHeading4 == "Canonical SMILES":
                                CanSmi = str(TOCHeading3[m].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String'))
                            elif TOCHeading4 == "Isomeric SMILES":
                                IsoSmi = str(TOCHeading3[m].get('Information')[0].get('Value').get('StringWithMarkup')[0].get('String'))
            if TOCHeading == 'Structures':
                for k in range(len(TOCHeading1)):
                    TOCHeading2 = TOCHeading1[k].get('TOCHeading')
                    if TOCHeading2 == '2D Structure':
                        boolean = str(TOCHeading1[k].get('Information')[0].get('Value').get('Boolean')[0])
                        if boolean != "":
                            TowD = 'Structure2D_CID_' + keyword[i] + '.sdf'
                    if TOCHeading2 == '3D Conformer':
                        boolean = str(TOCHeading1[k].get('Information')[0].get('Value').get('Number')[0])
                        if boolean != "":
                            ThreeD = 'Confermer3D_CID_' + keyword[i] + '.sdf'
        xx_info = [keyword[i], Name, MoleF, synon, MoleW, Sequence, SVG, IUPACC, TowD, ThreeD, IUPACN, Inchi, Inchi_key,
                   CanSmi, IsoSmi, TPSA, Comple, XLogP3, HAC, HBDC, HBAC, RBC, ExactM, FormC, PhyDes, MeltP, Solu, LogP,
                   Drug_in, Pharm, Distrub, Toxi, Metab, BioHalf, PDID, Infor, DrugB]
        sheet1.append(xx_info)
    wb.save('PubChem_info.xlsx')
